# First Python with LattePanda and its embedded Arduino Leonardo

Already made a working sketch to show LDR and thermistor.

We have already loaded the [*StandardFirmata*](https://github.com/firmata/arduino#usage) sketch ([old wiki](http://firmata.org/wiki/Main_Page)) into the on-board [Arduino (Leonardo model)](https://www.arduino.cc/en/Main/Arduino_BoardLeonardo) and will watch those same sensors from Python using [pyFirmata](https://pyfirmata.readthedocs.io/en/latest/).

Also added a PIR motion sensor on D9

In [1]:
import math
import time
from pyfirmata import Arduino, util

In [2]:
leonardo = Arduino("COM5")

In [3]:
it = util.Iterator(leonardo)
it.start()

In [4]:
raw_t0 = leonardo.get_pin("a:0:i")
raw_t1 = leonardo.get_pin("a:1:i")
raw_ldr = leonardo.get_pin("a:2:i")
raw_pir = leonardo.get_pin("d:9:i")

In [5]:
for item in (raw_t0, raw_t1, raw_ldr, raw_pir):
    item.enable_reporting()
    item.read()        # first read is usually None

In [13]:
def read_raw(pin, retries=5):
    """read signal from pyfirmata pin"""
    signal = None
    count = 0
    while signal is None and count < retries:
        count += 1
        signal = pin.read()
    return signal
        

def read_temperature(pin):
    """read temperature for thermistor"""
    C_KELVIN_OFFSET = 273.15
    OHM_REF = 10000
    
    # https://www.skyeinstruments.com/wp-content/uploads/Steinhart-Hart-Eqn-for-10k-Thermistors.pdf
    A = 0.001125308852122
    B = 0.000234711863267
    C = 0.000000085663516
    
    signal = read_raw(pin)
    if signal is None:
        return
    value = OHM_REF / (1/signal - 1)            # resistance, ohms
    
    # use Steinhart-Hart approximation to get temperature
    value = math.log(value)
    value = 1/(A + B*value + C*value*value*value) - C_KELVIN_OFFSET
    return value

In [15]:
print(read_temperature(raw_t0))
print(read_temperature(raw_t1))
print(read_raw(raw_ldr))
print(read_raw(raw_pir))

32.20144011403602
30.695129443104236
0.7937
False


In [16]:
t0 = time.time()
deadline = t0 + 60.0
while time.time() <= deadline:
    elapsed = time.time() - t0
    T0 = read_temperature(raw_t0)
    T1 = read_temperature(raw_t1)
    ldr = read_raw(raw_ldr)
    pir = read_raw(raw_pir)
    if None not in (ldr, T0, T1):
        print(
            "%8.3f" % elapsed,
            " T0=%.3fC" % T0,
            " T1=%.3fC" % T1,
            f" LDR={ldr}",
            f" PIR={pir}",
        )
        time.sleep(0.2)

   0.000  T0=32.582C  T1=31.165C  LDR=0.7889  PIR=False
   0.203  T0=32.582C  T1=31.165C  LDR=0.7937  PIR=False
   0.407  T0=32.582C  T1=31.165C  LDR=0.7947  PIR=False
   0.622  T0=32.582C  T1=31.165C  LDR=0.7908  PIR=False
   0.831  T0=32.582C  T1=31.165C  LDR=0.7889  PIR=False
   1.037  T0=32.582C  T1=31.165C  LDR=0.7937  PIR=False
   1.248  T0=32.582C  T1=31.165C  LDR=0.7928  PIR=False
   1.454  T0=32.582C  T1=31.262C  LDR=0.7928  PIR=False
   1.664  T0=32.582C  T1=31.165C  LDR=0.7937  PIR=False
   1.877  T0=32.494C  T1=31.262C  LDR=0.7928  PIR=False
   2.088  T0=32.582C  T1=31.262C  LDR=0.7898  PIR=False
   2.289  T0=32.582C  T1=31.262C  LDR=0.7937  PIR=False
   2.490  T0=32.680C  T1=31.262C  LDR=0.7908  PIR=False
   2.697  T0=32.680C  T1=31.262C  LDR=0.7908  PIR=False
   2.900  T0=32.680C  T1=31.165C  LDR=0.7918  PIR=False
   3.113  T0=32.680C  T1=31.262C  LDR=0.7947  PIR=False
   3.328  T0=32.778C  T1=31.165C  LDR=0.7937  PIR=False
   3.540  T0=32.778C  T1=31.262C  LDR=0.7879  PI

  30.537  T0=33.548C  T1=30.599C  LDR=0.782  PIR=False
  30.737  T0=33.548C  T1=30.599C  LDR=0.7859  PIR=False
  30.943  T0=33.448C  T1=30.599C  LDR=0.7849  PIR=False
  31.152  T0=33.548C  T1=30.695C  LDR=0.7889  PIR=False
  31.357  T0=33.548C  T1=30.599C  LDR=0.7849  PIR=False
  31.562  T0=33.548C  T1=30.599C  LDR=0.784  PIR=False
  31.771  T0=33.647C  T1=30.599C  LDR=0.783  PIR=False
  31.976  T0=33.647C  T1=30.599C  LDR=0.782  PIR=False
  32.187  T0=33.647C  T1=30.877C  LDR=0.7859  PIR=False
  32.393  T0=33.746C  T1=30.599C  LDR=0.783  PIR=False
  32.605  T0=33.746C  T1=30.695C  LDR=0.7859  PIR=False
  32.807  T0=33.746C  T1=30.695C  LDR=0.7879  PIR=False
  33.014  T0=33.746C  T1=30.695C  LDR=0.7879  PIR=False
  33.229  T0=33.746C  T1=30.695C  LDR=0.781  PIR=False
  33.431  T0=33.746C  T1=30.791C  LDR=0.782  PIR=False
  33.634  T0=33.845C  T1=30.791C  LDR=0.7879  PIR=False
  33.845  T0=33.845C  T1=30.791C  LDR=0.7879  PIR=False
  34.050  T0=33.845C  T1=30.791C  LDR=0.784  PIR=False


## Arduino sketch (for reference)

```
#include <math.h>

#define COMM_RATE 115200
#define LOOP_DELAY_MS 23
#define SMOOTHING 0.75
#define V_REF 5.2
#define OHM_REF 10000
#define C_KELVIN_OFFSET 273.15

// https://www.skyeinstruments.com/wp-content/uploads/Steinhart-Hart-Eqn-for-10k-Thermistors.pdf
#define A  0.001125308852122 
#define B  0.000234711863267 
#define C  0.000000085663516

#define LDR_THRESHHOLD 10


float ldr, thermistor;

float read_ldr() {
  int sig = analogRead(A1);
  int ref = analogRead(A2);
  // float value = V_REF * float(sig) / float(ref); // voltage, VDC
  float value = OHM_REF / (float(ref) / float(sig) - 1);  // resistance, ohms
  return(value);
}

float read_thermistor() {
  // NTC 10K with 10k resistor in series
  int sig = analogRead(A0);
  int ref = analogRead(A2);
  // float value = V_REF * float(sig) / float(ref); // voltage, VDC
  float value = OHM_REF / (float(ref) / float(sig) - 1);  // resistance, ohms
  // use Steinhart-Hart approximation to get temperature
  value = log(value);
  value = 1/(A + B*value + C*value*value*value) - C_KELVIN_OFFSET;
  return(value);
}

float smoothing(float value, float reading) {
  value *= SMOOTHING;
  value += reading * (1-SMOOTHING);
  return(value);
}

void setup() {
  Serial.begin(COMM_RATE);
  ldr = read_ldr();
  thermistor = read_thermistor();
}

void loop() {
  ldr = smoothing(ldr, read_ldr());
  thermistor = smoothing(thermistor, read_thermistor());

  Serial.print(micros() * 1.0e-6, 3);
  Serial.print("  thermistor (A0): ");
  Serial.print(thermistor);
  Serial.print("  LDR (A1): ");
  Serial.println(ldr, 0);

  if (ldr >= LDR_THRESHHOLD)
    digitalWrite(LED_BUILTIN, HIGH);
  else
    digitalWrite(LED_BUILTIN, LOW);

  delay(LOOP_DELAY_MS);
}
```